In [1]:
from sage.symbolic.function import SymbolicFunction
from sage.symbolic.expression import Expression
from sage.symbolic.ring import SR
from sage.symbolic.assumptions import assume 
import numpy as np
import sympy as sp
from typing import List
from sage.combinat.shuffle import ShuffleProduct
import itertools
from numpy import matrix 
from itertools import combinations as comb
import multiprocessing 
#from sage.parallel.multiprocessing_sage import Pool
from operator import add
import operator
from sys import getsizeof
import random
from numpy import array
from collections import Counter
import functools
import copy
import scipy 
import matplotlib.pyplot as plt
import networkx as nx
from multiprocessing import get_context
from sage.combinat.root_system.associahedron import Associahedra
from fractions import Fraction
from scipy.special import bernoulli, zeta 
import numbers
import decimal
from functools import reduce
from sympy import symbols, Symbol, var
import math
from sage.all import CartanMatrix, CartanType, var, Integer


In [2]:
def combineLikeTerms3(l):

    r=[]
    l2=copy.deepcopy(l)
    
    #print('l',l)
    temp2=[]

    for i in l2:

        if (IsInList(removeAnElementInList(0,i), temp2)==0): 
            t=0
            for j in l2:
                if removeAnElementInList(0,j)==removeAnElementInList(0,i):
                    t=t+j[0]
              
            s=i
            s[0]=t
            r.append(s)
            temp2.append(removeAnElementInList(0,i))
    
    r1=r
    r=[]
    for i in r1:
        if i[0]!=0:
            r.append(i)

    return r 

def removeAnElementInList(i, l):
    r=copy.deepcopy(l)
    r.pop(i)
    
    return r 

def IsInList(i,l):
    if (i in l)==True:
        r=1
    else:
        r=0
    return r 

In [3]:
def generate_symbols(base_name, rank):
    """
    Generate list of symbols base_name0 to base_name{rank}.
    """
    symbols = var([f"{base_name}{i}" for i in range(rank + 1)])  # Generate rank+1 variables (B0, B1, ..., B{rank})
    return list(symbols)

def multiply_two_polynomials(p1, p2):
    """Multiply two polynomials, each polynomial is a list of terms (monomials as lists with coefficients and exponents)."""
    result = []
    
    # Multiply each term in p1 with each term in p2
    for term1 in p1:
        for term2 in p2:
            # Multiply the coefficients and concatenate the exponents
            new_coefficient = term1[0] * term2[0]
            new_term = [new_coefficient] + term1[1:] + term2[1:]
            result.append(new_term)
    
    return result

def remove_first_element(term):
    """Remove the first element of a monomial (term)."""
    return term[1:]

def IndexOfAnElement(a, l):
    for i, x in enumerate(l):
        if a == x:
            return i + 1  # MuPAD uses 1-based indexing
    return 0

def IsABeforeB(a, b, l):
    i = IndexOfAnElement(a, l)
    j = IndexOfAnElement(b, l)
    return int(i <= j)

def InterchangeAandB(i, j, l):
    r = l[:]
    r[i], r[j] = r[j], r[i]
    return r

def NormalizationMonomial(L, l):
    i = 1  # corresponds to 2nd element, due to Sage 0-based indexing
    while i < len(L) - 1:
        y1 = L[i]
        y2 = L[i+1]
        if IsInList(y1, l) and IsInList(y2, l) and not IsABeforeB(y1, y2, l):
            L = InterchangeAandB(i, i+1, L)
            i = 1
        else:
            i += 1
    return L

def NormalizationPolynomial(poly, order_list):
    return [NormalizationMonomial(mono, order_list) for mono in poly]

def Multiplyq2(poly, q):
    return [[simplify(q * term[0])] + term[1:] for term in poly]

def UnionOfLists(l1, l2):
    return l1 + l2

def CommutatorQTwoLists(a1, a2, q):
    r1 = multiply_two_polynomials(a1, a2)
    r2 = multiply_two_polynomials(a2, a1)
    r2 = Multiplyq2(r2, -q)
    return UnionOfLists(r1, r2)

def MoveK1ToThebeginningToCoefficient(l, K1):
    """
    For each monomial term in list l (of the form [coeff, f1, f2, …]),
    pull any factor whose head is K1 into the coefficient.
    """
    from sage.all import SR
    r = []
    for term in l:
        coeff = term[0]
        new_factors = []
        for x in term[1:]:
            # detect if x is a K1‑factor (e.g. K1, K1^n, or K1*…)
            is_k1 = False
            if x == K1:
                is_k1 = True
            else:
                try:
                    head = x.operator()
                    if head == K1:
                        is_k1 = True
                except (AttributeError, TypeError):
                    pass
            if is_k1:
                coeff *= x
            else:
                new_factors.append(x)
            print(x, is_k1, head)
        r.append([coeff] + new_factors)
    return combineLikeTerms3(r)

def is_KK(var):
    return str(var).startswith('KK')

def is_E(var):
    return str(var).startswith('E')

def is_B(var):
    return str(var).startswith('B')

def is_F(var):
    return str(var).startswith('F')

def MoveK1ToThebeginningToCoefficient(l, K1):
    """
    Moves any factors starting with 'KK' from the monomial part into the coefficient.
    """
    r = []
    for term in l:
        coeff = term[0]
        new_factors = []
        for x in term[1:]:
            if is_KK(x):
                coeff *= x
            else:
                new_factors.append(x)
        r.append([coeff] + new_factors)
    return combineLikeTerms3(r)   

def quantum_group_relations(typ, rank, q, K, E1, F):
    """
    Return quantum group relations for U_q(g) of type (typ, rank).
    - Generators: K[i], Kinv[i], E1[i], F[i], indexed from 1.
    - Output: list of [LHS, RHS], where RHS is list of [coeff, monomial...].
    """
    # Cartan matrix and symmetrizer d_i
    C   = CartanMatrix([typ, rank])
    sym = CartanType([typ, rank]).symmetrizer()

    # q_i = q^{d_i}
    qq = [None] + [q**sym[i] for i in range(1, rank+1)]
    Kinv = [None] + [1 / K[i] for i in range(1, rank+1)]

    rels = []

    # 0) K_i K_j = K_j K_i (commute)
    for i in range(1, rank+1):
        for j in range(i+1, rank+1):
            rels.append([[K[i], K[j]], [[1, K[j], K[i]]]])
            rels.append([[Kinv[i], Kinv[j]], [[1, Kinv[j], Kinv[i]]]]) 
            
    # 1) K, K^{-1} with E, F: all solved for E, F on the left
    for i in range(1, rank + 1):
        for j in range(1, rank + 1):
            a = C[i - 1, j - 1]
            qia = qq[i] ** a

            # ---- E_j K_i = q^{-a_ij} K_i E_j ----
            rels.append([[K[i],    E1[j]],     [[ qia,  E1[j],    K[i]]]])
            # ---- E_j K_i^{-1} = q^{a_ij} K_i^{-1} E_j ----
            rels.append([[Kinv[i], E1[j]],  [[1/qia,   E1[j],    Kinv[i]  ]]])

            # ---- F_j K_i = q^{a_ij} K_i F_j ----
            rels.append([[K[i],    F[j]],     [[1/qia,    F[j],     K[i] ]]])
            # ---- F_j K_i^{-1} = q^{-a_ij} K_i^{-1} F_j ----
            rels.append([[Kinv[i], F[j]],  [[ qia, F[j],     Kinv[i] ]]])


    # 2) K_i K_i^{-1} = 1
    for i in range(1, rank+1):
        rels.append([[K[i],    Kinv[i]], [[1]]])
        rels.append([[Kinv[i], K[i]],    [[1]]])

    # 3) [E_i, F_i] = (K_i - K_i^{-1}) / (q_i - q_i^{-1}); and [E_i, F_j] = 0 if i ≠ j
    for i in range(1, rank+1):
        denom = qq[i] - 1/qq[i]
        rels.append([[E1[i], F[i]], [
            [1,        F[i],    E1[i]],
            [1/denom,  K[i]],
            [-1/denom, Kinv[i]]
        ]])
        for j in range(1, rank+1):
            if j != i:
                rels.append([[E1[i], F[j]], [[1, F[j], E1[i]]]])

    # 4) Serre relations and commutation
    for i in range(1, rank+1):
        for j in range(1, rank+1):
            if i == j:
                continue
            aij = C[i-1, j-1]
            if aij == 0:
                # commute
                rels.append([[E1[i], E1[j]], [[1, E1[j], E1[i]]]])
                rels.append([[F[i],  F[j]],  [[1,  F[j],  F[i]]]])
            else:
                m  = 1 - aij
                qi = qq[i]
                # Serre for E
                LHS_e = [E1[i]]*m + [E1[j]]
                RHS_e = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon = [E1[i]]*(m-k) + [E1[j]] + [E1[i]]*k
                    RHS_e.append([coeff] + mon)
                rels.append([LHS_e, RHS_e])
                # Serre for F
                LHS_f = [F[i]]*m + [F[j]]
                RHS_f = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon = [F[i]]*(m-k) + [F[j]] + [F[i]]*k
                    RHS_f.append([coeff] + mon)
                rels.append([LHS_f, RHS_f])

    return rels 

def quantum_group_relations_j_larger(typ, rank, q, K, E1, F):
    """
    Return quantum group relations for U_q(g) of type (typ, rank).
    - Generators: K[i], Kinv[i], E1[i], F[i], indexed from 1.
    - Output: list of [LHS, RHS], where RHS is list of [coeff, monomial...].
    """
    # Cartan matrix and symmetrizer d_i
    C   = CartanMatrix([typ, rank])
    sym = CartanType([typ, rank]).symmetrizer()

    # q_i = q^{d_i}
    qq = [None] + [q**sym[i] for i in range(1, rank+1)]
    Kinv = [None] + [1 / K[i] for i in range(1, rank+1)]

    rels = []

    # 0) K_i K_j = K_j K_i (commute)
    for i in range(1, rank+1):
        for j in range(i+1, rank+1):
            rels.append([[K[i], K[j]], [[1, K[j], K[i]]]])
            rels.append([[Kinv[i], Kinv[j]], [[1, Kinv[j], Kinv[i]]]]) 
            
    # 1) K, K^{-1} with E, F: all solved for E, F on the left
    for i in range(1, rank + 1):
        for j in range(1, rank + 1):
            a = C[i - 1, j - 1]
            qia = qq[i] ** a

            # ----  K_i E_j = q^{a_ij} E_j K_i =  ----
            rels.append([[K[i],    E1[j]],     [[ qia,  E1[j],    K[i]]]])
            # ---- K_i^{-1} E_j = q^{-a_ij} E_j K_i^{-1}  ----
            rels.append([[Kinv[i], E1[j]],  [[1/qia,   E1[j],    Kinv[i]  ]]])

            # ----K_i F_j = q^{-a_ij} F_j K_i ----
            rels.append([[K[i],    F[j]],     [[1/qia,    F[j],     K[i] ]]])
            # ---- K_i^{-1} F_j =  q^{a_ij} F_j K_i^{-1}  ----
            rels.append([[Kinv[i], F[j]],  [[ qia, F[j],     Kinv[i] ]]])

    # 2) K_i K_i^{-1} = 1
    for i in range(1, rank+1):
        rels.append([[K[i],    Kinv[i]], [[1]]])
        rels.append([[Kinv[i], K[i]],    [[1]]])

    # 3) [E_i, F_i] = (K_i - K_i^{-1}) / (q_i - q_i^{-1}); and [E_i, F_j] = 0 if i ≠ j
    for i in range(1, rank+1):
        denom = qq[i] - 1/qq[i]
        rels.append([[E1[i], F[i]], [
            [1,        F[i],    E1[i]],
            [1/denom,  K[i]],
            [-1/denom, Kinv[i]]
        ]])
        for j in range(1, rank+1):
            if j != i:
                rels.append([[E1[i], F[j]], [[1, F[j], E1[i]]]])

    # 4) Serre relations and commutation
    for i in range(1, rank+1):
        for j in range(1, rank+1):
            if i == j:
                continue
            aij = C[i-1, j-1]
            if aij == 0 and j>i:
                # commute
                rels.append([[E1[i], E1[j]], [[1, E1[j], E1[i]]]])
                rels.append([[F[i],  F[j]],  [[1,  F[j],  F[i]]]])
            elif j>i:
                m  = 1 - aij
                qi = qq[i]
                # Serre for E
                LHS_e = [E1[i]]*m + [E1[j]]
                RHS_e = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon = [E1[i]]*(m-k) + [E1[j]] + [E1[i]]*k
                    RHS_e.append([coeff] + mon)
                rels.append([LHS_e, RHS_e])
                # Serre for F
                LHS_f = [F[i]]*m + [F[j]]
                RHS_f = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon = [F[i]]*(m-k) + [F[j]] + [F[i]]*k
                    RHS_f.append([coeff] + mon)
                rels.append([LHS_f, RHS_f])

    return rels 

def quantum_group_relations_part(typ, rank, q, K, E1, F):
    """
    Return quantum group relations for U_q(g) of type (typ, rank).
    - Generators: K[i], Kinv[i], E1[i], F[i], indexed from 1.
    - Output: list of [LHS, RHS], where RHS is list of [coeff, monomial...].
    """
    # Cartan matrix and symmetrizer d_i
    C   = CartanMatrix([typ, rank])
    sym = CartanType([typ, rank]).symmetrizer()

    # q_i = q^{d_i}
    qq = [None] + [q**sym[i] for i in range(1, rank+1)]
    Kinv = [None] + [1 / K[i] for i in range(1, rank+1)]

    rels = []

    # 0) K_i K_j = K_j K_i (commute)
    for i in range(1, rank+1):
        for j in range(i+1, rank+1):
            rels.append([[K[i], K[j]], [[1, K[j], K[i]]]])
            rels.append([[Kinv[i], Kinv[j]], [[1, Kinv[j], Kinv[i]]]]) 
            
    # 1) K, K^{-1} with E, F: all solved for E, F on the left
    for i in range(1, rank + 1):
        for j in range(1, rank + 1):
            a = C[i - 1, j - 1]
            qia = qq[i] ** a

            # ----  K_i E_j = q^{a_ij} E_j K_i =  ----
            rels.append([[K[i],    E1[j]],     [[ qia,  E1[j],    K[i]]]])
            # ---- K_i^{-1} E_j = q^{-a_ij} E_j K_i^{-1}  ----
            rels.append([[Kinv[i], E1[j]],  [[1/qia,   E1[j],    Kinv[i]  ]]])

            # ----K_i F_j = q^{-a_ij} F_j K_i ----
            rels.append([[K[i],    F[j]],     [[1/qia,    F[j],     K[i] ]]])
            # ---- K_i^{-1} F_j =  q^{a_ij} F_j K_i^{-1}  ----
            rels.append([[Kinv[i], F[j]],  [[ qia, F[j],     Kinv[i] ]]])

    # 2) K_i K_i^{-1} = 1
    for i in range(1, rank+1):
        rels.append([[K[i],    Kinv[i]], [[1]]])
        rels.append([[Kinv[i], K[i]],    [[1]]])

    # 3) [E_i, F_i] = (K_i - K_i^{-1}) / (q_i - q_i^{-1}); and [E_i, F_j] = 0 if i ≠ j
    for i in range(1, rank+1):
        denom = qq[i] - 1/qq[i]
        rels.append([[E1[i], F[i]], [
            [1,        F[i],    E1[i]],
            [1/denom,  K[i]],
            [-1/denom, Kinv[i]]
        ]])
    return rels 

def simplify_coefficients(l):
    r = []
    for i in l:
        t1 = i[0].full_simplify()
        t1 = t1.factor()
        r.append([t1] + i[1:])
    return r

def RemoveZeroTerms(r):
    # Remove terms where the coefficient is 0
    return [term for term in r if term[0] != 0]

def q_number(n, q):
    return ((q^n - q^(-n)) / (q - q^(-1))).full_simplify()

def q_factorial(n, q):
    return simplify(prod(q_number(k, q) for k in range(1, n + 1)))

def q_binomial_sym(m, k, q):
    if k < 0 or k > m:
        return 0
    return simplify(q_factorial(m, q) / (q_factorial(k, q) * q_factorial(m - k, q)))

def multiply_polynomials(p1, p2):
    result = []
    for m1 in p1:
        for m2 in p2:
            coeff1 = m1[0]
            coeff2 = m2[0]
            new_coeff = coeff1 * coeff2
            new_tail = m1[1:] + m2[1:]
            result.append([new_coeff] + new_tail)
    return result

def change_pattern_in_polynomial(poly, pattern, replacement):
    result = []
    for monomial in poly:
        coeff = monomial[0]
        tail = monomial[1:]
        replaced = False

        for i in range(len(tail) - len(pattern) + 1):
            if tail[i:i + len(pattern)] == pattern:
                before = tail[:i]
                after = tail[i + len(pattern):]

                temp1 = [[1] + before]
                temp2 = [[1] + after]

                r1 = multiply_polynomials([[1] + before], replacement)
                r2 = multiply_polynomials(r1, [[1] + after])

                for term in r2:
                    term[0] *= coeff
                    result.append(term)
                replaced = True
                break

        if not replaced:
            result.append(monomial)

    return result 

def quantum_group_relations_affine(typ, rank, q, K, E, F):
    """
    Return quantum group relations for the untwisted affine U_q(g^{(1)}) of type (typ, rank).
    - Generators: K[i], Kinv[i], E[i], F[i], indexed from 0..rank.
    - Output: list of [LHS, RHS], where RHS is list of [coeff, monomial...].
    """
    # Cartan matrix of untwisted affine type g^{(1)}: use [typ, rank, 1]
    C   = CartanMatrix([typ, rank, 1])
    sym = CartanType([typ, rank, 1]).symmetrizer()

    # q_i = q^{d_i}
    qq = [q**sym[i] for i in range(0, rank+1)]
    Kinv = {i: 1/K[i] for i in range(0, rank+1)}

    rels = []

    # 0) K_i K_j = K_j K_i
    for i in range(0, rank+1):
        for j in range(i+1, rank+1):
            rels.append([[K[i], K[j]], [[1, K[j], K[i]]]])
            rels.append([[Kinv[i], Kinv[j]], [[1, Kinv[j], Kinv[i]]]])

    # 1) K, K^{-1} with E, F
    for i in range(0, rank+1):
        for j in range(0, rank+1):
            a = C[i, j]
            qia = qq[i]**a
            # E_j K_i = q^{-a_ij} K_i E_j 
            rels.append([[K[i], E[j]], [[qia, E[j], K[i]]]]) 
            rels.append([[Kinv[i], E[j]], [[1/qia, E[j], Kinv[i]]]]) 
            # F_j K_i = q^{a_ij} K_i F_j 
            rels.append([[K[i], F[j]], [[1/qia, F[j], K[i]]]]) 
            rels.append([[Kinv[i], F[j]], [[qia, F[j], Kinv[i]]]]) 

    # 2) K_i K_i^{-1} = 1
    for i in range(0, rank+1):
        rels.append([[K[i], Kinv[i]], [[1]]])
        rels.append([[Kinv[i], K[i]], [[1]]])

    # 3) [E_i, F_j]
    for i in range(0, rank+1):
        for j in range(0, rank+1):
            if i == j:
                denom = qq[i] - 1/qq[i]
                rels.append([[E[i], F[i]], [
                    [1,    F[i], E[i]],
                    [1/denom, K[i]],
                    [-1/denom, Kinv[i]]
                ]])
            else:
                rels.append([[E[i], F[j]], [[1, F[j], E[i]]]])

    # 4) Serre and commutation
    for i in range(0, rank+1):
        for j in range(0, rank+1):
            if i == j:
                continue
            aij = C[i, j]
            if aij == 0:
                rels.append([[E[i], E[j]], [[1, E[j], E[i]]]])
                rels.append([[F[i], F[j]], [[1, F[j], F[i]]]])
            else:
                m = 1 - aij
                qi = qq[i]
                # Serre E
                LHS_e = [E[i]]*m + [E[j]]
                RHS_e = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon = [E[i]]*(m-k) + [E[j]] + [E[i]]*k
                    RHS_e.append([coeff] + mon)
                rels.append([LHS_e, RHS_e])
                # Serre F
                LHS_f = [F[i]]*m + [F[j]]
                RHS_f = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon = [F[i]]*(m-k) + [F[j]] + [F[i]]*k
                    RHS_f.append([coeff] + mon)
                rels.append([LHS_f, RHS_f])

    return rels 

def quantum_group_relations_affine_j_larger(typ, rank, q, K, E, F):
    """
    Return quantum group relations for the untwisted affine U_q(g^{(1)}) of type (typ, rank),
    using only relations with j>i in the Serre‑commutation section.
    - Generators: K[i], Kinv[i], E[i], F[i], indexed from 0..rank.
    - Output: list of [LHS, RHS], where RHS is list of [coeff, monomial...].
    """
    # affine Cartan matrix and symmetrizer for g^{(1)}
    C   = CartanMatrix([typ, rank, 1])
    sym = CartanType([typ, rank, 1]).symmetrizer()

    # q_i = q^{d_i}
    qq   = [q**sym[i] for i in range(0, rank+1)]
    Kinv = {i: 1/K[i] for i in range(0, rank+1)}

    rels = []

    # 0) K_i K_j = K_j K_i  and same for inverses
    for i in range(0, rank+1):
        for j in range(i+1, rank+1):
            rels.append([[K[i],    K[j]],    [[1, K[j],    K[i]]]])
            rels.append([[Kinv[i], Kinv[j]], [[1, Kinv[j], Kinv[i]]]])

    # 1) K, K^{-1} with E, F
    for i in range(0, rank+1):
        for j in range(0, rank+1):
            a   = C[i, j]
            qia = qq[i]**a
            # E_j K_i = q^{-a_ij} K_i E_j 
            rels.append([[K[i], E[j]], [[qia, E[j], K[i]]]]) 
            rels.append([[Kinv[i], E[j]], [[1/qia, E[j], Kinv[i]]]]) 
            # F_j K_i = q^{a_ij} K_i F_j 
            rels.append([[K[i], F[j]], [[1/qia, F[j], K[i]]]]) 
            rels.append([[Kinv[i], F[j]], [[qia, F[j], Kinv[i]]]]) 
            
    # 2) K_i K_i^{-1} = 1
    for i in range(0, rank+1):
        rels.append([[K[i],    Kinv[i]], [[1]]])
        rels.append([[Kinv[i], K[i]],    [[1]]])

    # 3) [E_i, F_j]
    for i in range(0, rank+1):
        for j in range(0, rank+1):
            if i == j:
                denom = qq[i] - 1/qq[i]
                rels.append([[E[i], F[i]], [
                    [1,        F[i], E[i]],
                    [1/denom,  K[i]],
                    [-1/denom, Kinv[i]]
                ]])
            else:
                rels.append([[E[i], F[j]], [[1, F[j], E[i]]]])

    # 4) Serre and commutation only for j > i
    for i in range(0, rank+1):
        for j in range(i+1, rank+1):
            aij = C[i, j]
            if aij == 0:
                # commute
                rels.append([[E[i], E[j]], [[1, E[j], E[i]]]])
                rels.append([[F[i], F[j]], [[1, F[j], F[i]]]])
            else:
                m  = 1 - aij
                qi = qq[i]
                # Serre for E
                LHS_e = [E[i]]*m + [E[j]]
                RHS_e = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon   = [E[i]]*(m-k) + [E[j]] + [E[i]]*k
                    RHS_e.append([coeff] + mon)
                rels.append([LHS_e, RHS_e])

                # Serre for F
                LHS_f = [F[i]]*m + [F[j]]
                RHS_f = []
                for k in range(1, m+1):
                    coeff = -Integer((-1)**k) * q_binomial_sym(m, k, qi)
                    mon   = [F[i]]*(m-k) + [F[j]] + [F[i]]*k
                    RHS_f.append([coeff] + mon)
                rels.append([LHS_f, RHS_f])

    return rels


In [65]:
# finite type D4, check (8.8) in Lemma 8.6
typ='D'
rank=4 
q = var('q') 
K = generate_symbols('K', rank)
KK = generate_symbols('KK', rank)
E1 = generate_symbols('E', rank)
F = generate_symbols('F', rank)

In [66]:
# finite quantum group relations
relations_part=quantum_group_relations_j_larger(typ,rank,q,K,E1,F)
#relations_part

In [67]:
# check if [F_{2},[F_3, [\tilde{E}_4, [F_2, B_1]_q]_q]_q]_q = 0? It is identity (8.8) in Lemma 8.6
EE1=[[-q^(-2), KK[1], E1[1], 1/K[1]]]
EE4=[[-q^(-2), KK[4], E1[4], 1/K[4]]]
t1=CommutatorQTwoLists([[1, F[2]]], UnionOfLists([[1, F[1]]], EE1), q)
t2=CommutatorQTwoLists(EE4, t1, q)
t3=CommutatorQTwoLists([[1, F[3]]], t2, q)
t4=CommutatorQTwoLists([[1, F[2]]], t3, q)
r1=t4

In [69]:
# repeat running this cell until r1 cannot be simplified
r1=MoveK1ToThebeginningToCoefficient(r1,KK)
for ii in range(6):
    for i in relations_part:
        A1=i[0]
        C1=i[1]
        r1=change_pattern_in_polynomial(r1, A1, C1)
    r1=simplify_coefficients(r1)
for i in r1:
    print(i)

[-KK4*(q + 1)*(q - 1)/q, F2, F3, F2, F1, E4, 1/K4]
[KK4*(q + 1)*(q - 1)/q^2, F3, F2, F2, F1, E4, 1/K4]
[KK4*(q + 1)*(q - 1)/q, F2, F1, F2, F3, E4, 1/K4]
[-KK4*(q + 1)*(q - 1)/q^3, F3, F2, F1, F2, E4, 1/K4]
[KK4*(q + 1)*(q - 1)/q^2, F3, F1, F2, F2, E4, 1/K4]
[-KK4*(q + 1)*(q - 1)/q, F1, F2, F3, F2, E4, 1/K4]


In [71]:
for i0 in [(2, 3), (2, 1)]:
    i, j = i0 
    A1 = [ F[i], F[j], F[i] ]
    C1 = [
        [ (q + 1/q)**(-1), F[i], F[i], F[j] ],
        [ (q + 1/q)**(-1), F[j], F[i], F[i] ]
    ]
    # apply the pattern replacement
    r1 = change_pattern_in_polynomial(r1, A1, C1) 
    # normalize with respect to F1 and F3
    r1 = NormalizationPolynomial(r1, [ F[1], F[3] ])
    r1 = combineLikeTerms3(r1)
    r1=simplify_coefficients(r1) 
r1

[]

In [72]:
# finite type D4, check (8.7) in Lemma 8.6. Remark: after relabeling, this could be checked in type A3
typ='D'
rank=4 
q = var('q') 
K = generate_symbols('K', rank)
KK = generate_symbols('KK', rank)
E1 = generate_symbols('E', rank)
F = generate_symbols('F', rank)

In [73]:
# finite quantum group relations
relations_part=quantum_group_relations_j_larger(typ,rank,q,K,E1,F)
#relations_part

In [75]:
# check if [F_{3}, [\tilde{E}_4, \tilde{E}_2]_q]_q=0, for (8.7) in Lemma 8.6
EE2=[[-q^(-2), KK[2], E1[2], 1/K[2]]]
EE4=[[-q^(-2), KK[4], E1[4], 1/K[4]]]
t1=CommutatorQTwoLists(EE4, EE2, q)
t2=CommutatorQTwoLists([[1, F[3]]], t1, q)
r1=t2

In [77]:
# repeat running this cell until r1 cannot be simplified
r1=MoveK1ToThebeginningToCoefficient(r1,KK)
for ii in range(6):
    for i in relations_part:
        A1=i[0]
        C1=i[1]
        r1=change_pattern_in_polynomial(r1, A1, C1)
    r1=simplify_coefficients(r1)
r1

[]